<a href="https://colab.research.google.com/github/nicoelbert/vehicleroutingproblem/blob/main/vehicle_routing_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vehicle Routing Problem
vehicle routing problem on complete information using simmulated annealing

#Preparation

In [101]:
#import packages
import pandas as pd

import sys
#import classes


ModuleNotFoundError: ignored

##Data Import

In [41]:
#depot masterdata

#load Masterdata from csv
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/Depots.csv'
df_depots = pd.read_csv(url)


#load basetable
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/preparedJobs.csv'
df_jobs = pd.read_csv(url)


#next steps load data into classes
#create dictionary for each class

In [ ]:
#backup



"Evaluation of Constructionsites"
print(len(df_jobs['plz'].unique()))
print(len(df_jobs['lon'].unique()))
print(len(df_jobs['lat'].unique()))
#print(len(df_jobs[['lon','lat']]()))

df_sites = df_jobs[['lon','lat']].copy()

print(df_sites.shape)

df_sites.drop_duplicates(inplace=True)


print(df_sites.shape)
